In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import pandas as pd
pd.set_option('display.max_columns', None)

<center>
    <img src="https://logodownload.org/wp-content/uploads/2016/03/premier-league-5.png" width="75" alt="cognitiveclass.ai logo">
</center>

# Premier League API Retrieval, Data Collection and Visualization: Create & Visualize FPL Performance Metrics

<!-- Estimated time needed: **15** minutes -->

## Functions of notebook

This notebook was built to:

*   Retrieve data from Premier League API
-   Create attributes with fetched data to effectively collect and compile data.
*   Query data from attributres/functions to build visualization and statistics


In [ ]:
import sys
import os
sys.path.append(os.path.abspath('..'))
from src.functions.data_analysis import FPLDataAnalytics
from src.functions.data_visualization import NotebookDataVisualizer, FigureExporter
from src.functions.notebook_utils import create_search_widget

In [ ]:
fpl_data_attributes = FPLDataAnalytics()

In [ ]:
create_search_widget(list(fpl_data_attributes.helper_fns.__dict__.keys()))

In [ ]:
# FigureExporter(fpl_data_attributes)

In [ ]:
data_vis_ops = NotebookDataVisualizer(fpl_data_attributes)

---

### League Analysis

In [ ]:
# league_rank_data = fpl_data_attributes.helper_fns.get_rank_data(778049)

In [ ]:
# league_rank_data

---

## Performance Spread

In [ ]:
full_sum = data_vis_ops.display_tabular_summary()

In [ ]:
fpl_sum = data_vis_ops.build_player_tabular_summary(list(fpl_data_attributes.personal_team_data.keys()))

In [ ]:
names = [
    'sanchez',
    'konate',
    'konsa',
    'van de ven',
    'rogers',
    'palmer',
    'watkins'
]
values = [fpl_data_attributes.helper_fns.find_best_match(name) for name in names]
values += list(fpl_data_attributes.personal_team_data.keys())
custom_sum = data_vis_ops.build_player_tabular_summary(values)

---

## Exploratory Data Analysis

In [ ]:
from copy import deepcopy
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
latest_data_from_fpl = deepcopy(fpl_data_attributes.helper_fns.master_summary)

### Box Plot Visualization

In [ ]:
def generate_zipped_params(param_list, reference_dict):
    parsed_data = [{param: data[param] for param in param_list} for player_id, data in reference_dict.items()]
    result = []
    keys = list(parsed_data[0].keys())
    for d in parsed_data:
        for i in range(len(d[keys[0]])):
            if all(d[key][i][0] == d[keys[0]][i][0] for key in d.keys()):
                result.append(tuple(d[key][i][1] for key in d.keys()))
    return result

def generate_box_plot_for_params(y_param, x_param, data):
    
    matched_tuples = generate_zipped_params([y_param,x_param], data)
    
    reference_df = pd.DataFrame(matched_tuples, columns=[y_param, x_param])
    reference_df = reference_df.loc[reference_df[x_param] > 0]
    
    grouped = reference_df.groupby(x_param)[y_param].apply(list)

    # Plotting
    plt.figure(figsize=(20, 6))
    plt.boxplot(grouped.values)
    plt.xticks(range(1, len(grouped) + 1), grouped.index)
    plt.xlabel(x_param)
    plt.ylabel(y_param)
    plt.title(f"Box plots for every '{x_param}' value against attained '{y_param}'")

    # Add transparent rectangles
    if x_param == "total_points":
        plt.gca().add_patch(patches.Rectangle((0.5, plt.ylim()[0]), 3, plt.ylim()[1]-plt.ylim()[0],
                                              alpha=0.3, color='red'))
        plt.gca().add_patch(patches.Rectangle((3.5, plt.ylim()[0]), 2, plt.ylim()[1]-plt.ylim()[0],
                                              alpha=0.3, color='yellow'))
        plt.gca().add_patch(patches.Rectangle((5.5, plt.ylim()[0]), 4, plt.ylim()[1]-plt.ylim()[0],
                                              alpha=0.3, color='green'))
        plt.gca().add_patch(patches.Rectangle((9.5, plt.ylim()[0]), 22, plt.ylim()[1]-plt.ylim()[0],
                                             alpha=0.3, color='blue'))
    plt.show()

In [ ]:
latest_data_from_fpl[1].keys()

In [ ]:
generate_box_plot_for_params(
    y_param = 'bps', 
    x_param = 'total_points',
    data = latest_data_from_fpl
)

---